In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:

msft = yf.Ticker("AAPL")
print(msft)

yfinance.Ticker object <AAPL>


In [3]:
msft.history(period="5d")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-03-03,124.809998,125.709999,121.839996,122.059998,112430400,0,0
2021-03-04,121.750000,123.599998,118.620003,120.129997,177275300,0,0
2021-03-05,120.980003,121.940002,117.570000,121.419998,153590400,0,0
2021-03-08,120.930000,121.000000,116.209999,116.360001,153918600,0,0
2021-03-09,119.029999,122.059998,118.790001,121.084999,129525080,0,0


In [4]:
series = msft.history(period = "100d")
# series = pd.Series([1,2,3,4,5,6,7,8,9,10])

In [5]:
series.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-10-14,120.611374,122.634853,119.235809,120.800766,151062300,0.0,0
2020-10-15,118.338697,120.810728,117.770528,120.322304,112559200,0.0,0
2020-10-16,120.890469,121.159606,118.428401,118.637726,115393800,0.0,0
2020-10-19,119.574711,120.033233,115.288526,115.607498,120639300,0.0,0
2020-10-20,115.826793,118.597871,115.258624,117.132591,124423700,0.0,0


In [6]:
aapl = yf.download(tickers = ['AAPL'],
                  period='100d')

[*********************100%***********************]  1 of 1 completed


In [7]:

# note trail_size includes the current record. 
def compute_highest_high(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the hioghest high. """
    return series['High'].rolling(window=trail_size, min_periods=1).max()

In [8]:
def compute_lowest_low(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the lowest low. """
    return series['Low'].rolling(window=trail_size, min_periods=1).min()

In [9]:
def compute_avg_vol(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the average volume. """
    return series['Volume'].rolling(window=trail_size, min_periods=1).mean()

In [10]:
def compute_sma(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the simple moving average. """
    return series['Close'].rolling(window=trail_size, min_periods=1).mean()

In [11]:

def compute_sd(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the standard deviation. """
    return series['Close'].rolling(window=trail_size, min_periods=1).std()

In [12]:
def compute_willr(series):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series Williams %R marker. """
    return -100*((series['High']-series['Close'])/(series['High']-series['Low']))

In [13]:
def compute_atr(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series Average True Range. """
    
    # get a series of the previous day's closes
    prev_closes = series['Close'].shift(1).fillna(method='bfill')
    
    # make a df of the 3 differences we want to max
    triple_diffs = pd.DataFrame()
    triple_diffs['highlow'] = series['High'] - series['Low']
    triple_diffs['highclose'] = abs(series['High'] - prev_closes)
    triple_diffs['closelow'] = abs(prev_closes - series['Low'])
    
    true_ranges = triple_diffs.max(axis=1)
    
    return true_ranges.rolling(window=trail_size, min_periods=1).mean()

In [14]:
def compute_dmh(series):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series Directional Movement High. """
    return series['High'].diff(1).fillna(0)

In [15]:
def compute_dml(series):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series Directional Movement Low. """
    return series['Low'].diff(1).fillna(0)

In [16]:
def compute_ema(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the exponential moving average. """
    
    return series['Close'].ewm(span=trail_size).mean()

In [17]:
def compute_wma(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the weighted moving average. """
    
    weights = np.arange(1, trail_size+1)
    weights = weights[::-1]
    #should produce array [trailsize+1, trailsize, ..., 2, 1]
    
    #SUM ((close[t]*N) + (close[t-1] *(N-1)) + ... + (close[t-N])) 
    #wma = wma/(sum of the weights)
    wma = series['Close'].rolling(window=trail_size).apply(lambda prices: np.dot(prices, weights)/weights.sum(), raw=True)
    
    return wma

In [18]:
def compute_bbhigh(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the High Bollinger Bands. """
    #SMA[N] + SD[N]*2

    return series['SMA'] + series['SD']*2

In [19]:

def compute_bblow(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the Low Bollinger Bands. """
    #SMA[N] - SD[N]*2
    
    return series['SMA'] - series['SD']*2

In [20]:

def compute_perbhigh(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the Higher Limit Bollinger Bands. """
    #BBHIGH[N]/SD[N]
    return series['BBHIGH'].div(series['SD'])

In [21]:

def compute_perblow(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the Lower Limit Bollinger Bands. """
    #BBLOW[N]/SD[N]
    return series['BBLOW'].div(series['SD'])

In [22]:

def compute_trima(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the triangular moving average. """
    #(SMA[t-N] + ... + SMA[t])/N
    return (series['SMA'].rolling(window=trail_size).sum() / trail_size)

In [23]:

def compute_rsi(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the relative strength index. 
    This code was taken off stackoverflow. It produces warnings but
    does what we need it to do. Can't seem to figure out why it returns
    series rsi and RSI in the table"""
    
    change = pd.DataFrame()
    change = series['Close'].diff(1) # Calculate change

    # calculate gain / loss from every change
    series['Gain'] = np.select([change>0, change.isna()], 
                        [change, np.nan], 
                        default=0) 
    series['Loss'] = np.select([change<0, change.isna()], 
                        [-change, np.nan], 
                        default=0)

    # create avg_gain /  avg_loss columns with all nan
    series['Avg_gain'] = np.nan 
    series['Avg_loss'] = np.nan


    # keep first occurrence of rolling mean
    series['Avg_gain'][trail_size] = series['Gain'].rolling(window=trail_size).mean().dropna().iloc[0] 
    series['Avg_loss'][trail_size] = series['Loss'].rolling(window=trail_size).mean().dropna().iloc[0]
    
    #Looping through the pandas series
    for i in range(trail_size+1, series.shape[0]):
        series['Avg_gain'].iloc[i] = (series['Avg_gain'].iloc[i-1] * (trail_size - 1) + series['Gain'].iloc[i]) / trail_size
        series['Avg_loss'].iloc[i] = (series['Avg_loss'].iloc[i-1] * (trail_size - 1) + series['Loss'].iloc[i]) / trail_size

    # calculate rs and rsi
    series['rs'] = series['Avg_gain'] / series['Avg_loss']
    rsi = 100 - (100 / (1 + series['rs'] ))

    del series['Gain']
    del series['Loss']
    del series['Avg_gain']
    del series['Avg_loss']
    del series['rs']

    return rsi

In [24]:

def compute_dx(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the directional index. """
    #(abs(+DM - -DM)/(+DM + -DM))*100 
    
    num = abs(series['DMH'] - series['DML'])
    den = abs(series['DMH'] + series['DML'])

    return num / den * 100

In [25]:

def compute_positive_di(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the positive directional indicator. """
    #(+DM/ATR[N])*100 
    
    return (series['DMH'].div(series['ATR']).mul(100))

In [26]:
def compute_negative_di(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the negative directional indicator. """
    #(-DM/ATR[N])*100 
    
    return (series['DML'].div(series['ATR']).mul(100))

In [67]:

def compute_adx(series,trail_size = 20):
    """A function to take in a dataframe for one stock time series,
    and return a panda series Average Directional Index."""
    
    # iterative solution given recursive formula
    # init to zero. 
    adx_vals = np.zeros(len(series))
    for i in range(1, len(series)):
        adx_vals[i] = (adx_vals[i-1]*(trail_size-1)+series['DX'].values[i]) / trail_size
        
        #print(i,adx_vals[i],adx_vals[i-1],trail_size-1,trail_size,series['DX'].values[i])
    return adx_vals

In [68]:
a = compute_adx(series,20)
#print((a[55]*19+series['DX'].values[55])/20)

1 0.545454811680153 0.0 19 20 10.90909623360306
2 2.0527934034472133 0.545454811680153 19 20 30.692226647021357
3 4.30995666769501 2.0527934034472133 19 20 47.196058688403156
4 8.890387575042917 4.30995666769501 19 20 95.91857481465313
5 18.35519462227689 8.890387575042917 19 20 198.18652851972237
6 19.789212282291764 18.35519462227689 19 20 47.035547822574394
7 22.07754158403706 19.789212282291764 19 20 65.55579831719766
8 25.973676171634406 22.07754158403706 19 20 100.00023333598394
9 26.620609682989624 25.973676171634406 19 20 38.912346398738755
10 26.792417125096687 26.620609682989624 19 20 30.056758525130878
11 26.222032750988454 26.792417125096687 19 20 15.384729642932008
12 25.155095961060514 26.222032750988454 19 20 4.883296952429623
13 26.55817543012725 25.155095961060514 19 20 53.21668534239523
14 26.58164050359476 26.55817543012725 19 20 27.02747689947743
15 25.563441449182342 26.58164050359476 19 20 6.217659415346471
16 24.571817736967382 25.563441449182342 19 20 5.73096720

In [46]:
def compute_roc(series,trail_size = 20):
    """A function to take in a dataframe for one stock time series,
    and return a panda series Rate of Change."""
    return (series['Close']/(series['Close'].shift(trail_size).fillna(method='bfill'))-1)*10

In [47]:
def compute_macd(series,trail_size = 20):
    """A function to take in a dataframe for one stock time series,
    and return a panda series Moving Average Convergance Divergence."""
    
    return compute_ema(series, trail_size) - compute_ema(series, 2*trail_size)

In [48]:
def compute_cci(series,trail_size = 20):
    """A function to take in a dataframe for one stock time series,
    and return a panda series Commonodity Channel Index."""
    
    num = (series['High'] + series['Low'] + series['SMA'])/3
    
    return (num - series['SMA']) / (0.015*series['SD'])

In [49]:

def add_technical_markers(series, trail_size=20):
    series['HH'] = compute_highest_high(series, trail_size)
    series['LL'] = compute_lowest_low(series, trail_size)
    series['AV'] = compute_avg_vol(series, trail_size)
    series['SMA'] = compute_sma(series, trail_size)
    series['SD'] = compute_sd(series, trail_size)
    series['WILLR'] = compute_willr(series)
    series['ATR'] = compute_atr(series, trail_size)
    series['DMH'] = compute_dmh(series)
    series['DML'] = compute_dml(series)
    series['EMA'] = compute_ema(series, trail_size)
    series['WMA'] = compute_wma(series, trail_size)
    series['BBHIGH'] = compute_bbhigh(series, trail_size)
    series['BBLOW'] = compute_bblow(series, trail_size)
    series['PERBHIGH'] = compute_perbhigh(series, trail_size)
    series['PERBLOW'] = compute_perblow(series, trail_size)
    series['TRIMA'] = compute_trima(series, trail_size)
    series['RSI'] = compute_rsi(series, trail_size)
    series['DX'] = compute_dx(series, trail_size)
    series['PDI'] = compute_positive_di(series, trail_size)
    series['NDI'] = compute_negative_di(series, trail_size)
    series['ADX'] = compute_adx(series,trail_size = 20)
    series['ROC'] = compute_roc(series,trail_size = 20)
    series['MACD'] = compute_macd(series,trail_size = 20)
    series['CCI'] = compute_cci(series,trail_size = 20)
    
    return series

In [69]:
series = add_technical_markers(series, trail_size=3)

1 0.545454811680153 0.0 19 20 10.90909623360306
2 2.0527934034472133 0.545454811680153 19 20 30.692226647021357
3 4.30995666769501 2.0527934034472133 19 20 47.196058688403156
4 8.890387575042917 4.30995666769501 19 20 95.91857481465313
5 18.35519462227689 8.890387575042917 19 20 198.18652851972237
6 19.789212282291764 18.35519462227689 19 20 47.035547822574394
7 22.07754158403706 19.789212282291764 19 20 65.55579831719766
8 25.973676171634406 22.07754158403706 19 20 100.00023333598394
9 26.620609682989624 25.973676171634406 19 20 38.912346398738755
10 26.792417125096687 26.620609682989624 19 20 30.056758525130878
11 26.222032750988454 26.792417125096687 19 20 15.384729642932008
12 25.155095961060514 26.222032750988454 19 20 4.883296952429623
13 26.55817543012725 25.155095961060514 19 20 53.21668534239523
14 26.58164050359476 26.55817543012725 19 20 27.02747689947743
15 25.563441449182342 26.58164050359476 19 20 6.217659415346471
16 24.571817736967382 25.563441449182342 19 20 5.73096720

<ipython-input-23-32fbadae6c78>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series['Avg_gain'][trail_size] = series['Gain'].rolling(window=trail_size).mean().dropna().iloc[0]
<ipython-input-23-32fbadae6c78>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series['Avg_loss'][trail_size] = series['Loss'].rolling(window=trail_size).mean().dropna().iloc[0]
c:\users\david jiang\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [51]:
# for i in range(1, len(series)):
#     print(series['DX'][i])

In [71]:
series.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,HH,LL,AV,SMA,SD,WILLR,ATR,DMH,DML,EMA,WMA,BBHIGH,BBLOW,PERBHIGH,PERBLOW,TRIMA,RSI,DX,PDI,NDI,ADX,ROC,MACD,CCI
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-10-14,120.611374,122.634853,119.235809,120.800766,151062300,0.0,0,122.634853,119.235809,1.510623e+08,120.800766,NaN,-53.958900,3.399044,0.000000,0.000000,120.800766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
2020-10-15,118.338697,120.810728,117.770528,120.322304,112559200,0.0,0,122.634853,117.770528,1.318108e+08,120.561535,0.338324,-16.065528,3.219622,-1.824125,-1.465280,120.481791,NaN,121.238183,119.884887,358.349469,354.349469,NaN,NaN,10.909096,-56.656495,-45.510950,0.545455,-0.039608,-0.005981,-166.954649
2020-10-16,120.890469,121.159606,118.428401,118.637726,115393800,0.0,0,122.634853,117.770528,1.263384e+08,119.920265,1.136183,-92.335815,3.056816,0.348878,0.657873,119.428040,120.280772,122.192631,117.647899,107.546605,103.546605,120.427522,NaN,30.692227,11.413132,21.521505,2.052793,-0.179058,-0.036500,-4.939010
2020-10-19,119.574711,120.033233,115.288526,115.607498,120639300,0.0,0,121.159606,115.288526,1.161974e+08,118.189176,2.389194,-93.277310,3.505370,-1.126374,-3.139875,117.390417,118.974977,122.967563,113.410789,51.468230,47.468230,119.556992,0.000000,47.196059,-32.132803,-89.573275,4.309957,-0.429904,-0.109912,-9.827519
2020-10-20,115.826793,118.597871,115.258624,117.132591,124423700,0.0,0,121.159606,115.258624,1.201523e+08,117.125938,1.515125,-43.880554,3.605053,-1.435361,-0.029902,117.257346,117.376794,120.156188,114.095689,79.304483,75.304483,118.411793,30.579708,95.918575,-39.815267,-0.829442,8.890388,-0.303655,-0.117590,-5.799026


In [72]:
top100tech = ['AAPL',
'MSFT',
'TSM',
'NVDA',
'INTC',
'ASML',
'ADBE',
'CRM',
'AVGO',
'ORCL',
'CSCO',
'ACN',
'TXN',
'SHOP',
'QCOM',
'SAP',
'SNE',
'AMAT',
'INTU',
'NOW',
'SQ',
'IBM',
'MU',
'AMD',
'UBER',
'FIS',
'LRCX',
'FISV',
'SNOW',
'INFY',
'TEAM',
'ADSK',
'DELL',
'WDAY',
'VMW',
'ADI',
'NXPI',
'KLAC',
'CRWD',
'PLTR',
'DOCU',
'TEL',
'ERIC',
'MCHP',
'CDNS',
'CTSH',
'APH',
'SNPS',
'HPQ',
'PANW',
'OKTA',
'RNG',
'STM',
'MRVL',
'XLNX',
'WIT',
'MSI',
'SWKS',
'GLW',
'DDOG',
'ANSS',
'U',
'ZS',
'FTNT',
'STNE',
'ZBRA',
'KEYS',
'FTV',
'MXIM',
'COUP',
'GRMN',
'SPLK',
'HUBS',
'WORK',
'NET',
'FLT',
'CDW',
'CAJ',
'AFRM',
'PAYC',
'TER',
'VRSN',
'WDC',
'ZI',
'UMC',
'ANET',
'EPAM',
'UI',
'XM',
'GIB',
'TRMB',
'TYL',
'WIX',
'HPE',
'LYFT',
'LOGI',
'MPWR']

In [73]:
' '.join(top100tech)

'AAPL MSFT TSM NVDA INTC ASML ADBE CRM AVGO ORCL CSCO ACN TXN SHOP QCOM SAP SNE AMAT INTU NOW SQ IBM MU AMD UBER FIS LRCX FISV SNOW INFY TEAM ADSK DELL WDAY VMW ADI NXPI KLAC CRWD PLTR DOCU TEL ERIC MCHP CDNS CTSH APH SNPS HPQ PANW OKTA RNG STM MRVL XLNX WIT MSI SWKS GLW DDOG ANSS U ZS FTNT STNE ZBRA KEYS FTV MXIM COUP GRMN SPLK HUBS WORK NET FLT CDW CAJ AFRM PAYC TER VRSN WDC ZI UMC ANET EPAM UI XM GIB TRMB TYL WIX HPE LYFT LOGI MPWR'

In [74]:
data = yf.download(tickers = ' '.join(top100tech),
                  period='365d')

[*********************100%***********************]  97 of 97 completed


In [75]:
data.head(8)

Adj Close                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             \
                 AAPL         ACN        ADBE         ADI        ADSK AFRM       AMAT        AMD        ANET        ANSS        APH        ASML        AVGO        CAJ       CDNS         CDW        COUP         CRM       CRWD       CSCO       CTSH       DDOG       DELL       DOCU        EPAM      ERIC         FIS        FISV         FLT       FTNT        FTV        GIB        GLW       GRMN        HPE        HPQ        HUBS         IBM       INFY       INTC        INTU       KEYS        KLAC       LOGI        LRCX       LYFT       MCHP        MPWR       MRVL        MSFT         MSI         MU       MXIM        NET         NOW        NVDA        NXPI        OKTA       ORCL        PANW        PAYC PLTR       QCOM         RNG         SAP        SHOP        SNE SNOW        SNPS        SPLK         SQ        STM       STNE       SWKS        TEAM        TEL        TER       TRMB        TSM         TXN         TYL   U       UBER          UI       UMC         VMW        VRSN        WDAY   
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
2019-09-26  54.257603  187.634476  283.809998  109.860291  149.070007  NaN  51.181396  29.469999  244.009995  220.080002  47.590179  244.773651  262.158478  26.740000  66.580002  121.184364  135.059998  152.339996  58.299999  46.475861  58.491013  32.439999  52.150002  62.389999  183.639999  7.870853  131.770584  105.070000  292.700012  78.860001  56.544361  78.650002  27.010159  81.731773  13.897098  17.607441  154.210007  133.027023  10.726837  49.114510  266.713776  99.279999  157.878571  39.582912  236.674011  41.900002  90.507942  158.104004  24.693995  137.292786  168.779648  48.599998  56.524612  19.549999  257.989990  176.917313  106.669861  102.110001  52.590385  208.789993  212.300003  NaN  74.483719  126.720001  115.985573  313.140015  59.410000  NaN  140.740005  119.570000  60.799999  19.114094  35.639999  78.615158  128.179993  90.403496  58.789417  38.779999  44.942287  122.772499  263.190002 NaN  31.570000  114.249519  2.035685  147.360001  191.190002  174.440002   
2019-09-27  53.993587  186.9

In [76]:
df = pd.melt(data, ignore_index=False)
df.columns = ['Variable', 'Ticker', 'Value']
nice_df = df.reset_index().pivot(index=['Ticker', 'Date'], columns=['Variable'], values=['Value'])
nice_df.columns = nice_df.columns.droplevel()
nice_df.head(5)
df = nice_df.reset_index()

df.columns.name=None
df.head(10)
df.groupby('Ticker').mean().head()


,Adj Close,Close,High,Low,Open,Volume
Ticker,,,,,,
AAPL,93.482333,93.935630,95.140760,92.604671,93.890986,1.418702e+08
ACN,212.645503,214.743616,216.972465,212.441808,214.619150,2.183748e+06
ADBE,400.452822,400.452822,405.536822,394.532904,400.529863,2.936094e+06
ADI,119.234593,121.021781,122.612602,119.381671,121.081945,2.804604e+06
ADSK,219.027562,219.027562,222.395904,215.454301,218.925342,1.601050e+06


In [77]:
dataset = df.groupby('Ticker').apply(add_technical_markers)
dataset.head()


1 5.130746983145507 0.0 19 20 102.61493966291013
2 4.9513490241922025 5.130746983145507 19 20 1.5427878040794314
3 4.866899039635118 4.9513490241922025 19 20 3.262349333050523
4 5.370577156249359 4.866899039635118 19 20 14.940461371919922
5 5.268093725853249 5.370577156249359 19 20 3.320908548327158
6 5.733922756609222 5.268093725853249 19 20 14.584674340972711
7 6.0053040739233206 5.733922756609222 19 20 11.161549102891186
8 6.237585786633112 6.0053040739233206 19 20 10.650938328119146
9 13.521918185022454 6.237585786633112 19 20 151.92423375441996
10 13.99431711222663 13.521918185022454 19 20 22.969896729105965
11 14.191408780790686 13.99431711222663 19 20 17.93615048350775
12 16.762524503932994 14.191408780790686 19 20 65.61372324363684
13 28.702301641268132 16.762524503932994 19 20 255.55806725063576
14 28.15958401930426 28.702301641268132 19 20 17.84794920199073
15 29.14991021848047 28.15958401930426 19 20 47.96610800282847
16 29.192459790404847 29.14991021848047 19 20 30.00090165

291 61.21111839901634 63.272164318128276 19 20 22.05124593588976
292 78.41357243730516 61.21111839901634 19 20 405.2601991647928
293 77.04087853060614 78.41357243730516 19 20 50.95969430332489
294 74.22978845808139 77.04087853060614 19 20 20.81907708011103
295 72.93209024866641 74.22978845808139 19 20 48.27582426978215
296 70.15142157263168 72.93209024866641 19 20 17.318716727971733
297 70.19422793348511 70.15142157263168 19 20 71.00754878970051
298 67.29855514687013 70.19422793348511 19 20 12.280772201185634
299 70.21566111541574 67.29855514687013 19 20 125.64067451778239
300 69.33495294255772 70.21566111541574 19 20 52.601497658255205
301 66.58247747042249 69.33495294255772 19 20 14.285443499853098
302 65.2615395538266 66.58247747042249 19 20 40.16371913850464
303 63.71843327926026 65.2615395538266 19 20 34.3994140625
304 74.19314744202867 63.71843327926026 19 20 273.21271653462856
305 72.58246413512879 74.19314744202867 19 20 41.97948130403083
306 79.58717250121167 72.58246413512879

290 161.95059022928336 170.38142986008364 19 20 1.7646372440782327
291 155.96046434207676 161.95059022928336 19 20 42.14807248515114
292 149.01489591761566 155.96046434207676 19 20 17.04909585285502
293 143.40079048896934 149.01489591761566 19 20 36.732787344689065
294 136.2974263720445 143.40079048896934 19 20 1.3335081504723454
295 133.31465395637707 136.2974263720445 19 20 76.64197805869577
296 128.895432680748 133.31465395637707 19 20 44.93022844379575
297 125.70344619861767 128.895432680748 19 20 65.05570303814126
298 119.71478184608554 125.70344619861767 19 20 5.93015914797533
299 113.75283930237347 119.71478184608554 19 20 0.47593097184377836
300 114.83941470287793 113.75283930237347 19 20 135.4843473124631
301 110.0498199772007 114.83941470287793 19 20 19.04752018933339
302 105.47504292263702 110.0498199772007 19 20 18.554278885927015
303 102.45128600815178 105.47504292263702 19 20 44.99990463293216
304 102.09066539530096 102.45128600815178 19 20 95.23887375113533
305 99.403707

295 70.94504076207656 74.6367795850713 19 20 0.8020031251764972
296 67.43046204446894 70.94504076207656 19 20 0.6534664099239745
297 66.90911904272929 67.43046204446894 19 20 57.0036020096759
298 65.35131972685572 66.90911904272929 19 20 35.75313272525786
299 63.94739640939904 65.35131972685572 19 20 37.27285337772229
300 60.774072363971655 63.94739640939904 19 20 0.4809155008513432
301 72.59634445890833 60.774072363971655 19 20 297.21951426270505
302 71.46177153563718 72.59634445890833 19 20 49.904885993485344
303 70.913398504029 71.46177153563718 19 20 60.49431090347374
304 73.97306251094393 70.913398504029 19 20 132.1066786423275
305 70.3919164813658 73.97306251094393 19 20 2.3501419193811848
306 70.024961500038 70.3919164813658 19 20 63.05281685480989
307 67.40025649537166 70.024961500038 19 20 17.53086140671124
308 67.24920248181282 67.40025649537166 19 20 64.37917622419468
309 65.30250434191643 67.24920248181282 19 20 28.315239683885256
310 62.46620029856484 65.30250434191643 19 

286 114.35517839118604 119.82769093954523 19 20 10.377439972361376
287 112.06883895322123 114.35517839118604 19 20 68.62838963189022
288 108.11227711350332 112.06883895322123 19 20 32.93760215886297
289 103.90802363102293 108.11227711350332 19 20 24.02720746389564
290 100.67744441429375 103.90802363102293 19 20 39.296439296439296
291 99.04357916902549 100.67744441429375 19 20 68.00013950892857
292 97.32972190216489 99.04357916902549 19 20 64.76643383181339
293 92.83962223483114 97.32972190216489 19 20 7.527728555489927
294 96.49311635782423 92.83962223483114 19 20 165.9095046946932
295 96.5289319280511 96.49311635782423 19 20 97.20942776236134
296 94.61913433770965 96.5289319280511 19 20 58.3329801212224
297 97.11037293181042 94.61913433770965 19 20 144.44390621972485
298 93.27811868175081 97.11037293181042 19 20 20.465287930618445
299 96.34151349826823 93.27811868175081 19 20 154.54601501209908
300 94.23277446803414 96.34151349826823 19 20 54.16673289358659
301 94.98265434270513 94.23

...

285 nan nan 19 20 nan
286 nan nan 19 20 nan
287 nan nan 19 20 nan
288 nan nan 19 20 nan
289 nan nan 19 20 nan
290 nan nan 19 20 nan
291 nan nan 19 20 nan
292 nan nan 19 20 nan
293 nan nan 19 20 nan
294 nan nan 19 20 nan
295 nan nan 19 20 nan
296 nan nan 19 20 nan
297 nan nan 19 20 nan
298 nan nan 19 20 nan
299 nan nan 19 20 nan
300 nan nan 19 20 nan
301 nan nan 19 20 nan
302 nan nan 19 20 nan
303 nan nan 19 20 nan
304 nan nan 19 20 nan
305 nan nan 19 20 nan
306 nan nan 19 20 nan
307 nan nan 19 20 nan
308 nan nan 19 20 nan
309 nan nan 19 20 nan
310 nan nan 19 20 nan
311 nan nan 19 20 nan
312 nan nan 19 20 nan
313 nan nan 19 20 nan
314 nan nan 19 20 nan
315 nan nan 19 20 nan
316 nan nan 19 20 nan
317 nan nan 19 20 nan
318 nan nan 19 20 nan
319 nan nan 19 20 nan
320 nan nan 19 20 nan
321 nan nan 19 20 nan
322 nan nan 19 20 nan
323 nan nan 19 20 nan
324 nan nan 19 20 nan
325 nan nan 19 20 nan
326 nan nan 19 20 nan
327 nan nan 19 20 nan
328 nan nan 19 20 nan
329 nan nan 19 20 nan
330 nan na

281 41.56320359089104 43.47947840362006 19 20 5.15398214903989
282 40.026625717099954 41.56320359089104 19 20 10.831646115069269
283 38.071686178208175 40.026625717099954 19 20 0.9278349392643322
284 37.709238692642465 38.071686178208175 19 20 30.82273646689393
285 40.56056426729606 37.709238692642465 19 20 94.73575018571438
286 40.58923095577077 40.56056426729606 19 20 41.1338980367904
287 47.6673095566982 40.58923095577077 19 20 182.15080297431928
288 46.556658741091056 47.6673095566982 19 20 25.454293244555416
289 45.43791146915527 46.556658741091056 19 20 24.181713302375343
290 67.953042712138 45.43791146915527 19 20 495.7405363288098
291 71.90313225885902 67.953042712138 19 20 146.9548336465583
292 75.82279548196117 71.90313225885902 19 20 150.29639672090198
293 72.53371158391644 75.82279548196117 19 20 10.041117521066866
294 71.15413686690857 72.53371158391644 19 20 44.94221724375922
295 71.2373356552222 71.15413686690857 19 20 72.81811263318113
296 70.85219753041045 71.237335655

292 nan nan 19 20 20.754686802737627
293 nan nan 19 20 20.931140327531452
294 nan nan 19 20 22.531722340311493
295 nan nan 19 20 108.33518771656547
296 nan nan 19 20 20.7362223925964
297 nan nan 19 20 23.376649112464147
298 nan nan 19 20 5999.866488651535
299 nan nan 19 20 55.24605858907211
300 nan nan 19 20 45.046862664390446
301 nan nan 19 20 34.438552942309435
302 nan nan 19 20 86.75491836928032
303 nan nan 19 20 49.30553899869399
304 nan nan 19 20 2.0526651982132447
305 nan nan 19 20 91.63744726551597
306 nan nan 19 20 19.45391247288701
307 nan nan 19 20 43.4785132602458
308 nan nan 19 20 150.0
309 nan nan 19 20 11.961539093343262
310 nan nan 19 20 15.384605979540012
311 nan nan 19 20 190.56478835731303
312 nan nan 19 20 51.85897416756
313 nan nan 19 20 1.3861338272552435
314 nan nan 19 20 24.2235553912788
315 nan nan 19 20 53.333192048488954
316 nan nan 19 20 187.09707190716733
317 nan nan 19 20 446.67480481167826
318 nan nan 19 20 5.791711650143677
319 nan nan 19 20 26.6666525381

294 inf inf 19 20 37.572471863864614
295 inf inf 19 20 2.7905622941714046
296 inf inf 19 20 51.162287554989305
297 inf inf 19 20 44.18548226251843
298 inf inf 19 20 251.10794635742502
299 inf inf 19 20 30.486596226161538
300 inf inf 19 20 34.25010825816211
301 inf inf 19 20 307.95378119390153
302 inf inf 19 20 461.9078067778876
303 inf inf 19 20 59.43839981718
304 inf inf 19 20 5.098912196074632
305 inf inf 19 20 29.03225806451613
306 inf inf 19 20 51.567329912067464
307 inf inf 19 20 9.202563106432574
308 inf inf 19 20 12.668765149105585
309 inf inf 19 20 8.046106032650254
310 inf inf 19 20 18.609380151922224
311 inf inf 19 20 15.29985245503614
312 inf inf 19 20 24.579290644595318
313 inf inf 19 20 13.935451531945922
314 inf inf 19 20 18.32449230670906
315 inf inf 19 20 200.2867043076003
316 inf inf 19 20 84.98586508286434
317 inf inf 19 20 48.816484904299664
318 inf inf 19 20 21.428874181809373
319 inf inf 19 20 2.5162664824091667
320 inf inf 19 20 41.77789362996221
321 inf inf 19 20

,Ticker,Date,Adj Close,Close,High,Low,Open,Volume,HH,LL,AV,SMA,SD,WILLR,ATR,DMH,DML,EMA,WMA,BBHIGH,BBLOW,PERBHIGH,PERBLOW,TRIMA,RSI,DX,PDI,NDI,ADX,ROC,MACD,CCI
0,AAPL,2019-09-26,54.257603,54.972500,55.235001,54.707500,55.000000,75334000.0,55.235001,54.7075,7.533400e+07,54.972500,NaN,-49.763163,0.527500,0.000000,0.000000,54.972500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
1,AAPL,2019-09-27,53.993587,54.705002,55.240002,54.320000,55.134998,101408000.0,55.240002,54.3200,8.837100e+07,54.838751,0.189150,-58.152032,0.723751,0.005001,-0.387501,54.832063,NaN,55.217050,54.460452,291.922543,287.922543,NaN,NaN,102.614940,0.690993,-53.540614,5.130747,-0.048660,-0.003344,-13.804507
2,AAPL,2019-09-30,55.264339,55.992500,56.145000,55.197498,55.224998,103909600.0,56.145000,54.3200,9.355053e+07,55.223334,0.679413,-16.094967,0.962500,0.904999,0.877499,55.258169,NaN,56.582159,53.864509,83.281004,79.281004,NaN,NaN,1.542788,94.025822,91.168664,4.951349,0.185547,0.017622,29.300829
3,AAPL,2019-10-01,55.417320,56.147499,57.055000,56.049999,56.267502,139223200.0,57.055000,54.3200,1.049687e+08,55.454375,0.721980,-90.298533,0.987500,0.910000,0.852501,55.514905,NaN,56.898334,54.010416,78.808784,74.808784,NaN,NaN,3.262349,92.151865,86.329190,4.866899,0.213743,0.030339,67.599601
4,AAPL,2019-10-02,54.028126,54.740002,55.895000,54.482498,55.764999,138449200.0,57.055000,54.3200,1.116648e+08,55.311501,0.702144,-81.769693,1.123000,-1.160000,-1.567501,55.327462,NaN,56.715789,53.907212,80.775101,76.775101,NaN,NaN,14.940461,-103.294702,-139.581532,5.370577,-0.042294,0.007063,-7.769926


In [78]:
dataset.tail()


,Ticker,Date,Adj Close,Close,High,Low,Open,Volume,HH,LL,AV,SMA,SD,WILLR,ATR,DMH,DML,EMA,WMA,BBHIGH,BBLOW,PERBHIGH,PERBLOW,TRIMA,RSI,DX,PDI,NDI,ADX,ROC,MACD,CCI
35400,ZS,2021-03-03,187.830002,187.830002,202.119995,187.460007,202.110001,2702900.0,230.880997,187.460007,1993580.00,213.773999,10.667837,-97.476157,10.941150,-9.880005,-14.809998,208.804111,217.155808,235.109673,192.438325,22.039114,18.039114,212.356899,NaN,19.967567,-90.301339,-135.360521,inf,-1.332657,3.362655,-79.091315
35401,ZS,2021-03-04,180.899994,180.899994,190.960007,175.679993,184.330002,3171400.0,230.880997,175.679993,2032355.00,212.340999,12.945482,-65.837720,11.147601,-11.159988,-11.780014,206.146576,217.420666,238.231963,186.450034,18.402711,14.402711,212.709449,NaN,2.702814,-100.111126,-105.673091,inf,-1.367628,1.902265,-99.634927
35402,ZS,2021-03-05,175.369995,175.369995,185.539993,165.570007,185.539993,3940900.0,230.880997,165.570007,2167605.00,210.421999,15.347455,-50.926416,11.800350,-5.420013,-10.109985,203.215473,217.103713,241.116909,179.727088,15.710546,11.710546,212.892724,NaN,30.199435,-45.930954,-85.675299,inf,-1.795556,0.379665,-100.970769
35403,ZS,2021-03-08,167.839996,167.839996,181.699997,166.886002,175.759995,2729100.0,230.880997,165.570007,2203170.00,207.974998,17.959589,-93.560179,12.135550,-3.839996,1.315994,199.846380,216.265142,243.894176,172.055821,13.580165,9.580165,212.894324,NaN,204.278382,-31.642541,10.844125,inf,-2.257588,-1.282315,-83.352561
35404,ZS,2021-03-09,181.429993,181.429993,182.789993,175.000000,178.714996,2824687.0,230.880997,165.570007,2286269.35,206.252998,18.794443,-17.458303,12.524550,1.089996,8.113998,198.092439,215.349237,243.841884,168.664113,12.974148,8.974148,212.763574,NaN,76.314712,8.702878,64.784751,inf,-1.595405,-2.075345,-64.695250
